### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

from concepts import static_concepts, linear_regression, generate_static_concept_datasets
from policy import ConvNet, ResNet
from utils import concept_folder_setup_and_score

import tensorflow as tf

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

model_name = "net"
session_name = "falcon"
board_size = 7
board_name = f'{board_size}x{board_size}'
resnet = True

model_type = "resnet" if resnet else "convnet"

agents_to_sample = [0, 10, 20, 60, 100, 500]

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.one_eye

CONCEPT_NAME = static_concepts.one_eye.__name__

BINARY = True

CASES_TO_SAMPLE = 200 # 25000

# Load the models
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, model_path)
    else:
        model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [3]:
positive_cases, negative_cases = generate_static_concept_datasets(CASES_TO_SAMPLE, agents, board_size, CONCEPT_FUNC, binary=BINARY)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases for concept 'one_eye': 495it [00:29, 16.64it/s]                         


In [4]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (200, 5, 7, 7)
Negative cases:  (200, 5, 7, 7)


In [6]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [5]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  6


In [ ]:
from env import gogame
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [9]:
# Positions to consider are 80% of the total positions
if BINARY:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000 #10000
else:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0])
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000

epochs = 10

Positions to consider: 320


In [10]:
# First test if the concept can be regressed form the inputs
name = "input"
if BINARY:
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
else:
    all_cases = np.array(positive_cases)
    all_labels = negative_cases

all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs,
    verbose=1
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, name, score)

Epoch 1/10


2024-03-01 13:44:51.454821: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


10/10 [==============================] - 1s 56ms/step - loss: 1.1178 - val_loss: 1.1400
Epoch 2/10
10/10 [==============================] - 0s 20ms/step - loss: 1.0947 - val_loss: 1.1203
Epoch 3/10
10/10 [==============================] - 0s 23ms/step - loss: 1.0742 - val_loss: 1.1015
Epoch 4/10
10/10 [==============================] - 0s 20ms/step - loss: 1.0544 - val_loss: 1.0842
Epoch 5/10
10/10 [==============================] - 0s 21ms/step - loss: 1.0366 - val_loss: 1.0681
Epoch 6/10
10/10 [==============================] - 0s 22ms/step - loss: 1.0201 - val_loss: 1.0532
Epoch 7/10
10/10 [==============================] - 0s 20ms/step - loss: 1.0044 - val_loss: 1.0397
Epoch 8/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9908 - val_loss: 1.0272
Epoch 9/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9781 - val_loss: 1.0156
Epoch 10/10
10/10 [==============================] - 0s 3ms/step
-0.13124999999999998
3/3 [=============================

In [11]:
from tqdm import tqdm
epochs_to_look_at = agents_to_sample

bar = tqdm(total=len(epochs_to_look_at), desc="Epochs")

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    if BINARY:
        all_cases = np.concatenate([positive_cases, negative_cases])
        all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    else:
        all_cases = np.array(positive_cases)
        all_labels = negative_cases

    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs,
            verbose=1
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
        
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, epoch, concept_presence_per_layer)

    bar.update()

Epochs:   0%|          | 0/6 [00:00<?, ?it/s]

Getting activation outputs: 100%|██████████| 13/13 [00:00<00:00, 20.86it/s]


Performing regression for layer 0
Epoch 1/10


2024-03-01 13:45:00.941086: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


10/10 [==============================] - 1s 68ms/step - loss: 1.2621 - val_loss: 1.2482
Epoch 2/10
10/10 [==============================] - 0s 22ms/step - loss: 1.2391 - val_loss: 1.2266
Epoch 3/10
10/10 [==============================] - 0s 21ms/step - loss: 1.2183 - val_loss: 1.2056
Epoch 4/10
10/10 [==============================] - 0s 31ms/step - loss: 1.1975 - val_loss: 1.1856
Epoch 5/10
10/10 [==============================] - 0s 21ms/step - loss: 1.1778 - val_loss: 1.1663
Epoch 6/10
10/10 [==============================] - 0s 21ms/step - loss: 1.1584 - val_loss: 1.1478
Epoch 7/10
10/10 [==============================] - 0s 20ms/step - loss: 1.1399 - val_loss: 1.1298
Epoch 8/10
10/10 [==============================] - 0s 22ms/step - loss: 1.1219 - val_loss: 1.1123
Epoch 9/10
10/10 [==============================] - 0s 20ms/step - loss: 1.1044 - val_loss: 1.0951
Epoch 10/10
10/10 [==============================] - 0s 3ms/step
0.11250000000000004
3/3 [==============================

...

10/10 [==============================] - 1s 71ms/step - loss: 1.2588 - val_loss: 1.2307
Epoch 2/10
10/10 [==============================] - 0s 23ms/step - loss: 1.2303 - val_loss: 1.2044
Epoch 3/10
10/10 [==============================] - 0s 22ms/step - loss: 1.2049 - val_loss: 1.1793
Epoch 4/10
10/10 [==============================] - 0s 22ms/step - loss: 1.1793 - val_loss: 1.1562
Epoch 5/10
10/10 [==============================] - 0s 21ms/step - loss: 1.1563 - val_loss: 1.1345
Epoch 6/10
10/10 [==============================] - 0s 20ms/step - loss: 1.1347 - val_loss: 1.1138
Epoch 7/10
10/10 [==============================] - 0s 21ms/step - loss: 1.1133 - val_loss: 1.0945
Epoch 8/10
10/10 [==============================] - 0s 21ms/step - loss: 1.0939 - val_loss: 1.0756
Epoch 9/10
10/10 [==============================] - 0s 21ms/step - loss: 1.0746 - val_loss: 1.0579
Epoch 10/10
10/10 [==============================] - 0s 3ms/step
0.5562499999999999
3/3 [==============================]

2024-03-01 13:47:39.894950: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


10/10 [==============================] - 1s 70ms/step - loss: 0.9897 - val_loss: 0.9636
Epoch 2/10
10/10 [==============================] - 0s 23ms/step - loss: 0.9857 - val_loss: 0.9593
Epoch 3/10
10/10 [==============================] - 0s 23ms/step - loss: 0.9820 - val_loss: 0.9549
Epoch 4/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9779 - val_loss: 0.9509
Epoch 5/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9745 - val_loss: 0.9466
Epoch 6/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9707 - val_loss: 0.9426
Epoch 7/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9675 - val_loss: 0.9384
Epoch 8/10
10/10 [==============================] - 0s 21ms/step - loss: 0.9637 - val_loss: 0.9348
Epoch 9/10
10/10 [==============================] - 0s 20ms/step - loss: 0.9606 - val_loss: 0.9309
Epoch 10/10
10/10 [==============================] - 0s 3ms/step
-0.0625
3/3 [==============================] - 0s 6ms/s

Epochs: 100%|██████████| 6/6 [02:43<00:00, 27.67s/it]

The presence of one_eye in resblock 7 is 0
